# Problem Set #2

MACS 30000, Dr. Evans

Haihao Guo

## Exercise 1

### A

In [1]:
import timeit
import warnings
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
warnings.filterwarnings("ignore")

df1 = pd.read_csv('data/Auto.csv',na_values="?")
df1.dropna(inplace=True)
tf = df1['mpg']>=df1['mpg'].median()
df1['mpg_high'] = np.where(tf,1,0)

df1['orgn1'] = 0
df1.orgn1[df1['origin']==1] = 1
df1['orgn2'] = 0
df1.orgn2[df1['origin']==2] = 1

y = df1['mpg_high'].values
X = df1[['cylinders', 'displacement', 'horsepower', 'weight','acceleration', 'year', 'orgn1', 'orgn2']].values
start_time = timeit.default_timer()
mse = []
for i in range(100):
    j = i+1
    X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.35, random_state=j)
    LR = LogisticRegression(solver='lbfgs', max_iter=2000, n_jobs=1)
    LR.fit(X_train, y_train)
    y_pred = LR.predict(X_test)
    mse_i = ((y_test - y_pred) ** 2).mean()
    mse.append(mse_i)
avmse = np.array(mse).mean()

elapsed_time = timeit.default_timer() - start_time
print('The Elapsed time is', elapsed_time, 'seconds')

print('The Average error rate is', avmse)

The Elapsed time is 3.306393001 seconds
The Average error rate is 0.10057971014492752


### B

In [2]:
import multiprocessing
num_cores = multiprocessing.cpu_count()
print('Number of available cores is', num_cores)

Number of available cores is 8


In [3]:
import multiprocessing
from dask import compute, delayed
import dask.multiprocessing
import time
from statistics import mean
num_cores = multiprocessing.cpu_count()
print('Number of available cores is', num_cores)

def doublecheck(bootstrap, seed, Data):
    X, y = Data
    X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.35, random_state=seed+bootstrap)
    LogReg = LogisticRegression()
    LogReg.fit(X_train, y_train)
    y_pred = LogReg.predict(X_test)
    err_rate_now = (y_test == y_pred).mean()
    return err_rate_now
time_start2 = time.clock()
err_rate_2 = []
for i in range(100):
    err_rate_2.append(delayed(doublecheck)(i,100,[X,y]))
err_rate_2_avg = delayed(mean)(err_rate_2)
err_rate_2_avg = compute(err_rate_2_avg, num_workers=num_cores)[0]
time_end2 = time.clock()
print('average error rate =', err_rate_2_avg)
print('computation time =',time_end2-time_start2,'seconds')

Number of available cores is 8
average error rate = 0.8986231884057971
computation time = 1.8585689999999992 seconds
